# Building Input Functions with tf.estimator 读书笔记

[tf官网原文](https://www.tensorflow.org/get_started/input_fn)

input_fn 的目的是把feature传入Extimator中的train, evaluate和predict中。可以在input_fn中做feature engineering 或者 pre-processing

In [1]:
import tensorflow as tf


## input_fn的基本形式
``` python
def my_input_fn():

    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels
```

回传值一定是feature_cols和labels。

**feature_cols**: A dict containing key/value pairs that map feature column names to Tensors (or SparseTensors) containing the corresponding feature data.

**labels**:A Tensor containing your label (target) values: the values your model aims to predict.

feature colums的官方解释：

A **FeatureColumn** represents a single feature in your data. A **FeatureColumn** may represent a quantity like 'height', or it may represent a category like 'eye_color' where the value is drawn from a set of discrete possibilities like {'blue', 'brown', 'green'}.

In the case of both continuous features like 'height' and categorical features like 'eye_color', a single value in the data might get transformed into a sequence of numbers before it is input into the model. The FeatureColumn abstraction lets you manipulate the feature as a single semantic unit in spite of this fact. You can specify transformations and select features to include without dealing with specific indices in the tensors you feed into the model.

自己的理解，由于有的时候要对某一个特定的feature做preprocessing，在pandas里面可以直接做到拿column的动作，应该tf也是支持这一属性的，于是称为feature column。

## numpy pandas资料处理

如果资料室pandas或者numpy，可以利用一下方法产生input_fn:
```python
import numpy as np

my_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_data)},
    y=np.array(y_data),
    ...)

import pandas as pd

my_input_fn = tf.extimator.inputs.pandas_input_fn(
    x=pd.DataFrame({"x": x_data}),
    y=pd.Series(y_data),
    ...)
```

## 将input_fn传入model

直接将function object传入input_fn中。
```python
classifier.train(input_fn=my_input_fn, steps=2000)
```
因为是function object，意味着我们无法限定这个function的输入参数，建议使用lambda来传入参数，这样对于train evaluate的input_fn并不需要重新命名。
```python
classifier.train(input_fn=lambda: my_input_fn(training_set), steps=2000)
classifier.evaluate(input_fn=lambda: my_input_fn(test_set), steps=2000)
```
如果需要更多的参数来控制function可以采用以下方法从numpy或pandas获取data set。

```python
import pandas as pd

def get_input_fn_from_pandas(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pdDataFrame(...),
      y=pd.Series(...),
      num_epochs=num_epochs,
      shuffle=shuffle)
      
import numpy as np

def get_input_fn_from_numpy(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.numpy_input_fn(
      x={...},
      y=np.array(...),
      num_epochs=num_epochs,
      shuffle=shuffle)
```

[tf.estimator.inputs.pandas_input_fn api](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn)

[ tf.estimator.inputs.numpy_input_fn api](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn)

# Boston House Values DNN

实作input_fn在DNN上。

[itertools 介绍](https://docs.python.org/3/library/itertools.html)

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd

tf.logging.set_verbosity(tf.logging.INFO)

定义columns，并利用pandas从csv内读取资料。

skipinitialspace : boolean, default False
Skip spaces after delimiter.

在delimiter后为什么会有spaces？自己测试True or False好像没有太大的区别。

In [3]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("./data/boston/boston_train.csv",
                           skipinitialspace=True,
                           skiprows=1,
                           names=COLUMNS)
test_set = pd.read_csv("./data/boston/boston_test.csv",
                      skipinitialspace=True,
                      skiprows=1,
                      names=COLUMNS)
prediction_set = pd.read_csv("./data/boston/boston_predict.csv",
                            skipinitialspace=True,
                            skiprows=1,
                            names=COLUMNS)

### 建构FeatureColumn

需要产生一个NumericColum的list

建構get_input_fn，让他return一个input function object，这样可以输入一些参数。

**num_epochs**：在training的时候设定为None，让train函数的steps控制次数。而在evaluate的时候设定为1，这样的话input_fn只跑一次。

In [4]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                     hidden_units=[10,10],
                                     model_dir="/tmp/boston_model")

# Building the input_fn

def get_input_fn(data_set, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
        y=pd.Series(data_set[LABEL].values),
        num_epochs=num_epochs,
        shuffle=shuffle)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_model_dir': '/tmp/boston_model', '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000}


设定5000次steps，并传入training_set。

In [5]:
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/boston_model/model.ckpt.
INFO:tensorflow:step = 1, loss = 49120.5
INFO:tensorflow:global_step/sec: 289.785
INFO:tensorflow:step = 101, loss = 15093.1 (0.348 sec)
INFO:tensorflow:global_step/sec: 309.621
INFO:tensorflow:step = 201, loss = 9873.31 (0.322 sec)
INFO:tensorflow:global_step/sec: 244.16
INFO:tensorflow:step = 301, loss = 10366.8 (0.409 sec)
INFO:tensorflow:global_step/sec: 270.095
INFO:tensorflow:step = 401, loss = 7410.75 (0.372 sec)
INFO:tensorflow:global_step/sec: 247.095
INFO:tensorflow:step = 501, loss = 9121.24 (0.405 sec)
INFO:tensorflow:global_step/sec: 301.696
INFO:tensorflow:step = 601, loss = 8376.78 (0.330 sec)
INFO:tensorflow:global_step/sec: 307.18
INFO:tensorflow:step = 701, loss = 6631.06 (0.328 sec)
INFO:tensorflow:global_step/sec: 292.642
INFO:tensorflow:step = 801, loss = 8016.91 (0.339 sec)
INFO:tensorflow:global_step/sec: 311.454
INFO:tensorflow:step = 901, loss

### 评估模型
利用test_set和model.evaluate来评估模型的好坏程度。

In [6]:
ev = regressor.evaluate(input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

INFO:tensorflow:Starting evaluation at 2017-08-24-07:19:54
INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-08-24-07:19:54
INFO:tensorflow:Saving dict for global step 5000: average_loss = 12.192, global_step = 5000, loss = 1219.2
Loss: 1219.203491


### 模型预测

利用model.predict来预测prediction_set。

In [7]:
y = regressor.predict(
    input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
Predictions: [array([ 34.39196396], dtype=float32), array([ 18.23705101], dtype=float32), array([ 22.64715195], dtype=float32), array([ 36.05002975], dtype=float32), array([ 14.84117413], dtype=float32), array([ 18.36227608], dtype=float32)]
